In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [71]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (SB, SQ), (MB, SQ), (MB, SQ), (SB, MQ), (MB, LQ), (MB, SQ), (SB, MQ),
     (SB, SQ), (MB, MQ), (LB, LQ), (SB, MQ), (MB, SQ), (MB, MQ), (MB, MQ),
      (MB, MQ), (SB, LQ), (SB, SQ), (MB, SQ), (SB, SQ), (MB, MQ)
 ]

In [77]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [78]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [10]:
print(len(queries_list))
print(len(combined_dataset))

20
2180000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


-----------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 681.04it/s]


Build Time: 1.61 sec, Search Time: 0.03 sec

1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 205.41it/s]


Build Time: 27.01 sec, Search Time: 0.10 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 44.67 sec, Search Time: 0.41 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 64.11it/s]


Build Time: 5.78 sec, Search Time: 0.31 sec

19: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 62.88it/s]

Build Time: 45.73 sec, Search Time: 1.58 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 1.61 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 27.01 sec, Search: 0.10 sec
Data:  100000 points, Queries:   20 => Build: 33.13 sec, Search: 0.14 sec
Data:   10000 points, Queries:  100 => Build: 3.23 sec, Search: 0.58 sec
Data:  100000 points, Queries:  500 => Build: 37.05 sec, Search: 3.93 sec
Data:  100000 points, Queries:   20 => Build: 37.47 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 3.62 sec, Search: 0.84 sec
Data:   10000 points, Queries:   20 => Build: 3.69 sec, Search: 0.14 sec
Data:  100000 points, Queries:  100 => Build: 38.62 sec, Search: 0.93 sec
Data: 1000000 points, Queries:  500 => Build: 419.64 sec, Search: 7.17 sec
Data:   10000 points, Queries:  100 => Build: 4.30 sec, Search: 1.17 sec
Data:  100000 points, Queries:   20 => Build: 43.54 sec, Search: 0.30 sec
Data:  100000 points, Queries:  100 =

**----------CV Run 2-----------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 769.00it/s]


Build Time: 4.07 sec, Search Time: 0.03 sec

1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 241.15it/s]


Build Time: 26.92 sec, Search Time: 0.08 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 45.35 sec, Search Time: 0.37 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 39.92it/s]


Build Time: 4.49 sec, Search Time: 0.50 sec

19: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 67.70it/s]

Build Time: 45.58 sec, Search Time: 1.46 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 4.07 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 26.92 sec, Search: 0.08 sec
Data:  100000 points, Queries:   20 => Build: 31.87 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 3.33 sec, Search: 0.57 sec
Data:  100000 points, Queries:  500 => Build: 34.67 sec, Search: 3.18 sec
Data:  100000 points, Queries:   20 => Build: 36.82 sec, Search: 0.14 sec
Data:   10000 points, Queries:  100 => Build: 3.60 sec, Search: 0.71 sec
Data:   10000 points, Queries:   20 => Build: 3.99 sec, Search: 0.17 sec
Data:  100000 points, Queries:  100 => Build: 38.64 sec, Search: 0.87 sec
Data: 1000000 points, Queries:  500 => Build: 423.45 sec, Search: 6.41 sec
Data:   10000 points, Queries:  100 => Build: 4.39 sec, Search: 1.19 sec
Data:  100000 points, Queries:   20 => Build: 44.29 sec, Search: 0.26 sec
Data:  100000 points, Queries:  100 =

**---------CV Analysis-------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 923.51',
    'Total Time: 924.74'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 924.1250 sec, Std Dev: 0.8697 sec, CV: 0.09%


--------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

-----------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1397.57it/s]


Build Time: 1.99 sec, Search Time: 0.01 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 611.95it/s]


Build Time: 21.33 sec, Search Time: 0.03 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 24.28 sec, Search Time: 0.03 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 634.54it/s]


Build Time: 2.10 sec, Search Time: 0.03 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 783.06it/s]

Build Time: 28.88 sec, Search Time: 0.13 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 1.99 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 21.33 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 22.23 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 2.22 sec, Search: 0.10 sec
Data:  100000 points, Queries:  500 => Build: 20.75 sec, Search: 0.62 sec
Data:  100000 points, Queries:   20 => Build: 23.99 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 4.27 sec, Search: 0.11 sec
Data:   10000 points, Queries:   20 => Build: 0.97 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 24.84 sec, Search: 0.12 sec
Data: 1000000 points, Queries:  500 => Build: 234.60 sec, Search: 0.89 sec
Data:   10000 points, Queries:  100 => Build: 2.10 sec, Search: 0.13 sec
Data:  100000 points, Queries:   20 => Build: 26.93 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => B

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 725.43it/s]


Build Time: 1.75 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 404.81it/s]


Build Time: 22.75 sec, Search Time: 0.05 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 23.51 sec, Search Time: 0.02 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1118.93it/s]


Build Time: 3.05 sec, Search Time: 0.02 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 767.10it/s]

Build Time: 27.96 sec, Search Time: 0.13 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 1.75 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 22.75 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 21.50 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 2.87 sec, Search: 0.10 sec
Data:  100000 points, Queries:  500 => Build: 22.69 sec, Search: 0.61 sec
Data:  100000 points, Queries:   20 => Build: 26.87 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 2.27 sec, Search: 0.12 sec
Data:   10000 points, Queries:   20 => Build: 1.60 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 24.08 sec, Search: 0.12 sec
Data: 1000000 points, Queries:  500 => Build: 229.03 sec, Search: 0.80 sec
Data:   10000 points, Queries:  100 => Build: 4.66 sec, Search: 0.11 sec
Data:  100000 points, Queries:   20 => Build: 27.02 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => B

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Vp tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 778.23it/s]


Build Time: 1.40 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 416.46it/s]


Build Time: 23.64 sec, Search Time: 0.05 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 28.59 sec, Search Time: 0.03 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 801.69it/s]


Build Time: 1.41 sec, Search Time: 0.02 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 760.33it/s]

Build Time: 24.75 sec, Search Time: 0.13 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 1.40 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 23.64 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 22.18 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 2.27 sec, Search: 0.10 sec
Data:  100000 points, Queries:  500 => Build: 26.04 sec, Search: 0.52 sec
Data:  100000 points, Queries:   20 => Build: 20.77 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 2.01 sec, Search: 0.14 sec
Data:   10000 points, Queries:   20 => Build: 1.29 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 25.17 sec, Search: 0.10 sec
Data: 1000000 points, Queries:  500 => Build: 239.73 sec, Search: 1.05 sec
Data:   10000 points, Queries:  100 => Build: 2.44 sec, Search: 0.16 sec
Data:  100000 points, Queries:   20 => Build: 26.12 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => B

**------VP MWV Analysis---------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 527.21',
    'Total Time: 521.20',
    'Total Time: 528.31'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 525.5733 sec, Std Dev: 3.8271 sec, CV: 0.73%


----------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

--------

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 2252.04it/s]

Build Time: 0.02 sec, Search Time: 0.01 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 2045.35it/s]


Build Time: 0.25 sec, Search Time: 0.01 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.89 sec, Search Time: 0.02 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 769.77it/s]


Build Time: 6.42 sec, Search Time: 0.03 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 801.44it/s]

Build Time: 6.81 sec, Search Time: 0.12 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.02 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.25 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.45 sec, Search: 0.01 sec
Data:   10000 points, Queries:  100 => Build: 0.51 sec, Search: 0.08 sec
Data:  100000 points, Queries:  500 => Build: 0.87 sec, Search: 0.32 sec
Data:  100000 points, Queries:   20 => Build: 1.00 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 1.03 sec, Search: 0.08 sec
Data:   10000 points, Queries:   20 => Build: 1.02 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 1.35 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 5.34 sec, Search: 0.60 sec
Data:   10000 points, Queries:  100 => Build: 4.69 sec, Search: 0.11 sec
Data:  100000 points, Queries:   20 => Build: 5.12 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 5.2

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 2948.96it/s]

Build Time: 0.01 sec, Search Time: 0.01 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 1936.88it/s]


Build Time: 0.21 sec, Search Time: 0.01 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.52 sec, Search Time: 0.02 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 667.15it/s]


Build Time: 6.43 sec, Search Time: 0.03 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 748.03it/s]

Build Time: 7.05 sec, Search Time: 0.13 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.01 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.21 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.44 sec, Search: 0.01 sec
Data:   10000 points, Queries:  100 => Build: 0.45 sec, Search: 0.06 sec
Data:  100000 points, Queries:  500 => Build: 0.74 sec, Search: 0.35 sec
Data:  100000 points, Queries:   20 => Build: 0.98 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 0.99 sec, Search: 0.08 sec
Data:   10000 points, Queries:   20 => Build: 1.12 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 1.36 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 4.59 sec, Search: 0.54 sec
Data:   10000 points, Queries:  100 => Build: 4.54 sec, Search: 0.11 sec
Data:  100000 points, Queries:   20 => Build: 4.89 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 5.2

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 4389.87it/s]

Build Time: 0.01 sec, Search Time: 0.00 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 1851.95it/s]


Build Time: 0.20 sec, Search Time: 0.01 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.31 sec, Search Time: 0.03 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 548.73it/s]


Build Time: 6.96 sec, Search Time: 0.04 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 583.27it/s]

Build Time: 6.88 sec, Search Time: 0.17 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.01 sec, Search: 0.00 sec
Data:  100000 points, Queries:   20 => Build: 0.20 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.42 sec, Search: 0.01 sec
Data:   10000 points, Queries:  100 => Build: 0.44 sec, Search: 0.06 sec
Data:  100000 points, Queries:  500 => Build: 0.73 sec, Search: 0.49 sec
Data:  100000 points, Queries:   20 => Build: 1.26 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 1.01 sec, Search: 0.08 sec
Data:   10000 points, Queries:   20 => Build: 1.03 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 1.33 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 4.52 sec, Search: 0.58 sec
Data:   10000 points, Queries:  100 => Build: 4.71 sec, Search: 0.11 sec
Data:  100000 points, Queries:   20 => Build: 4.98 sec, Search: 0.02 sec
Data:  100000 points, Queries:  100 => Build: 5.2

**--------Ball Tree Analysis------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 73.75',
    'Total Time: 71.88'
    'Total Time: 72.33'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 72.8150 sec, Std Dev: 1.3223 sec, CV: 1.82%


-------------

**----------QuickSelect Algorithm-----------**

In [13]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

------------------------------------------

**--------KD hybrid------------------**

In [14]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

------------------------

**-------kd hybrid sqrt run 1---------**

In [15]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0246 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2330.36it/s]

Total Search Time: 0.017477 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.4738 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 328.26it/s]


...

Total Search Time: 1.594672 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 123504.35it/s]


Insert Time: 0.8123 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.61it/s]

Total Search Time: 5.735840 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0246 sec, Total Search Time: 0.017477 sec, Total Time: 0.0421 sec
Points: 110,000, Build Time: 0.4738 sec, Total Search Time: 0.088337 sec, Total Time: 0.5622 sec
Points: 210,000, Insert Time: 0.8566 sec, Total Search Time: 0.158423 sec, Total Time: 1.0150 sec
Points: 220,000, Insert Time: 0.0779 sec, Total Search Time: 0.542383 sec, Total Time: 0.6202 sec
Points: 320,000, Build Time: 1.6556 sec, Total Search Time: 4.352137 sec, Total Time: 6.0078 sec
Points: 420,000, Insert Time: 0.8198 sec, Total Search Time: 0.309865 sec, Total Time: 1.1296 sec
Points: 430,000, Insert Time: 0.0915 sec, Total Search Time: 1.099441 sec, Total Time: 1.1909 sec
Points: 440,000, Insert Time: 0.1010 sec, Total Search Time: 0.314820 sec, Total Time: 0.4158 sec
Points: 540,000, Insert Time: 0.8171 sec, Total Search Time: 1.233494 sec, Total Time: 2.0506 sec
Points: 1,540,000, Build Time: 8.8374 sec, Total 

**-----------kd hybrid sqrt run 2---------**

In [20]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0265 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2420.33it/s]

Total Search Time: 0.013980 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.8130 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 333.74it/s]


...

Total Search Time: 1.709376 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 113256.69it/s]


Insert Time: 0.8861 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 18.91it/s]

Total Search Time: 5.962842 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0265 sec, Total Search Time: 0.013980 sec, Total Time: 0.0405 sec
Points: 110,000, Build Time: 0.8130 sec, Total Search Time: 0.090499 sec, Total Time: 0.9035 sec
Points: 210,000, Insert Time: 0.7824 sec, Total Search Time: 0.169824 sec, Total Time: 0.9522 sec
Points: 220,000, Insert Time: 0.0995 sec, Total Search Time: 0.534471 sec, Total Time: 0.6340 sec
Points: 320,000, Build Time: 1.5910 sec, Total Search Time: 4.450101 sec, Total Time: 6.0411 sec
Points: 420,000, Insert Time: 0.9233 sec, Total Search Time: 0.313871 sec, Total Time: 1.2372 sec
Points: 430,000, Insert Time: 0.0784 sec, Total Search Time: 1.143320 sec, Total Time: 1.2217 sec
Points: 440,000, Insert Time: 0.0911 sec, Total Search Time: 0.346773 sec, Total Time: 0.4379 sec
Points: 540,000, Insert Time: 0.7912 sec, Total Search Time: 1.268540 sec, Total Time: 2.0598 sec
Points: 1,540,000, Build Time: 9.0010 sec, Total 

**-------kd hybrid sqrt run 3-------------**

In [25]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0264 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2522.97it/s]

Total Search Time: 0.014300 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.8016 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 308.79it/s]


...

Total Search Time: 1.650140 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 126174.54it/s]


Insert Time: 0.7954 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.34it/s]

Total Search Time: 5.855520 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0264 sec, Total Search Time: 0.014300 sec, Total Time: 0.0407 sec
Points: 110,000, Build Time: 0.8016 sec, Total Search Time: 0.093273 sec, Total Time: 0.8949 sec
Points: 210,000, Insert Time: 0.7919 sec, Total Search Time: 0.169879 sec, Total Time: 0.9618 sec
Points: 220,000, Insert Time: 0.0986 sec, Total Search Time: 0.538369 sec, Total Time: 0.6370 sec
Points: 320,000, Build Time: 1.7581 sec, Total Search Time: 4.479515 sec, Total Time: 6.2376 sec
Points: 420,000, Insert Time: 0.7836 sec, Total Search Time: 0.320972 sec, Total Time: 1.1046 sec
Points: 430,000, Insert Time: 0.0759 sec, Total Search Time: 1.109861 sec, Total Time: 1.1858 sec
Points: 440,000, Insert Time: 0.0879 sec, Total Search Time: 0.327651 sec, Total Time: 0.4156 sec
Points: 540,000, Insert Time: 0.7996 sec, Total Search Time: 1.310671 sec, Total Time: 2.1103 sec
Points: 1,540,000, Build Time: 8.9542 sec, Total 

**------KD hybrid sqrt analysis-------**

In [29]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 109.7098',
    'Total Time:: 111.4053',
    'Total Time:: 110.4836'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 110.5329 sec, Std Dev: 0.8488 sec, CV: 0.77%


-----------------------------------------

**------KD hybrid log run 1--------**

In [16]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0264 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2493.49it/s]


Total Search Time: 0.013611 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.7387 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 311.61it/s]


Total Search Time: 0.092017 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.624000 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 112677.59it/s]


Insert Time: 0.8904 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.49it/s]

Total Search Time: 5.790024 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0264 sec, Total Search Time: 0.013611 sec, Total Time: 0.0400 sec
Points: 110,000, Build Time: 0.7387 sec, Total Search Time: 0.092017 sec, Total Time: 0.8308 sec
Points: 210,000, Insert Time: 0.8195 sec, Total Search Time: 0.167783 sec, Total Time: 0.9872 sec
Points: 220,000, Insert Time: 0.1160 sec, Total Search Time: 0.524114 sec, Total Time: 0.6402 sec
Points: 320,000, Insert Time: 0.8266 sec, Total Search Time: 2.910631 sec, Total Time: 3.7373 sec
Points: 420,000, Insert Time: 0.8052 sec, Total Search Time: 0.326434 sec, Total Time: 1.1316 sec
Points: 430,000, Insert Time: 0.0738 sec, Total Search Time: 0.935258 sec, Total Time: 1.0090 sec
Points: 440,000, Insert Time: 0.0907 sec, Total Search Time: 0.330902 sec, Total Time: 0.4216 sec
Points: 540,000, Insert Time: 0.8131 sec, Total Search Time: 1.139444 sec, Total Time: 1.9526 sec
Points: 1,540,000, Build Time: 8.9689 sec, Total

**-------KD Hybrid Run 2-----------**

In [21]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0339 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2304.31it/s]


Total Search Time: 0.014573 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.7782 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 335.40it/s]


Total Search Time: 0.088616 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.570481 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 111087.91it/s]


Insert Time: 0.9034 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.25it/s]

Total Search Time: 5.863806 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0339 sec, Total Search Time: 0.014573 sec, Total Time: 0.0485 sec
Points: 110,000, Build Time: 0.7782 sec, Total Search Time: 0.088616 sec, Total Time: 0.8669 sec
Points: 210,000, Insert Time: 0.8084 sec, Total Search Time: 0.173054 sec, Total Time: 0.9814 sec
Points: 220,000, Insert Time: 0.0925 sec, Total Search Time: 0.532237 sec, Total Time: 0.6248 sec
Points: 320,000, Insert Time: 0.9263 sec, Total Search Time: 3.062686 sec, Total Time: 3.9890 sec
Points: 420,000, Insert Time: 0.8383 sec, Total Search Time: 0.314462 sec, Total Time: 1.1527 sec
Points: 430,000, Insert Time: 0.0826 sec, Total Search Time: 0.952737 sec, Total Time: 1.0354 sec
Points: 440,000, Insert Time: 0.0758 sec, Total Search Time: 0.351409 sec, Total Time: 0.4272 sec
Points: 540,000, Insert Time: 0.7906 sec, Total Search Time: 1.158549 sec, Total Time: 1.9491 sec
Points: 1,540,000, Build Time: 8.9917 sec, Total

**------KD Hybrid Run 3------**

In [26]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0221 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2185.16it/s]


Total Search Time: 0.015058 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.8174 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 327.57it/s]


Total Search Time: 0.089461 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.551128 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 120455.71it/s]


Insert Time: 0.8321 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.39it/s]

Total Search Time: 5.787570 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0221 sec, Total Search Time: 0.015058 sec, Total Time: 0.0372 sec
Points: 110,000, Build Time: 0.8174 sec, Total Search Time: 0.089461 sec, Total Time: 0.9068 sec
Points: 210,000, Insert Time: 0.9203 sec, Total Search Time: 0.165717 sec, Total Time: 1.0860 sec
Points: 220,000, Insert Time: 0.0957 sec, Total Search Time: 0.564458 sec, Total Time: 0.6602 sec
Points: 320,000, Insert Time: 0.7879 sec, Total Search Time: 2.976909 sec, Total Time: 3.7648 sec
Points: 420,000, Insert Time: 0.7842 sec, Total Search Time: 0.342763 sec, Total Time: 1.1270 sec
Points: 430,000, Insert Time: 0.0906 sec, Total Search Time: 0.915172 sec, Total Time: 1.0058 sec
Points: 440,000, Insert Time: 0.0757 sec, Total Search Time: 0.340931 sec, Total Time: 0.4167 sec
Points: 540,000, Insert Time: 0.7940 sec, Total Search Time: 1.132444 sec, Total Time: 1.9265 sec
Points: 1,540,000, Build Time: 8.9620 sec, Total

**----------kd hybrid log analysis----------**

In [30]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 107.6988',
    'Total Time:: 107.6107',
    'Total Time:: 106.3956'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 107.2350 sec, Std Dev: 0.7283 sec, CV: 0.68%


----------------------------

**--------kd hybrid Log Ratio Run 1---------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0261 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2311.74it/s]


Total Search Time: 0.014892 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.4972 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 334.78it/s]


Total Search Time: 0.086393 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.616997 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 112589.33it/s]


Insert Time: 0.8913 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.22it/s]

Total Search Time: 5.888893 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0261 sec, Total Search Time: 0.014892 sec, Total Time: 0.0410 sec
Points: 110,000, Build Time: 0.4972 sec, Total Search Time: 0.086393 sec, Total Time: 0.5836 sec
Points: 210,000, Insert Time: 0.8406 sec, Total Search Time: 0.172505 sec, Total Time: 1.0131 sec
Points: 220,000, Insert Time: 0.0846 sec, Total Search Time: 0.550403 sec, Total Time: 0.6350 sec
Points: 320,000, Insert Time: 0.8636 sec, Total Search Time: 2.868285 sec, Total Time: 3.7319 sec
Points: 420,000, Insert Time: 0.8108 sec, Total Search Time: 0.352438 sec, Total Time: 1.1632 sec
Points: 430,000, Insert Time: 0.0869 sec, Total Search Time: 0.911764 sec, Total Time: 0.9987 sec
Points: 440,000, Insert Time: 0.0754 sec, Total Search Time: 0.373208 sec, Total Time: 0.4486 sec
Points: 540,000, Insert Time: 0.7701 sec, Total Search Time: 1.109617 sec, Total Time: 1.8797 sec
Points: 1,540,000, Build Time: 9.7313 sec, Total

**------KD Hybrid Log Ratio Run 2---------**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0246 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2521.22it/s]


Total Search Time: 0.013049 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.8177 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 348.94it/s]


Total Search Time: 0.084817 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.649520 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 107115.98it/s]


Insert Time: 0.9364 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.21it/s]

Total Search Time: 5.886494 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0246 sec, Total Search Time: 0.013049 sec, Total Time: 0.0376 sec
Points: 110,000, Build Time: 0.8177 sec, Total Search Time: 0.084817 sec, Total Time: 0.9025 sec
Points: 210,000, Insert Time: 0.8241 sec, Total Search Time: 0.172217 sec, Total Time: 0.9963 sec
Points: 220,000, Insert Time: 0.0889 sec, Total Search Time: 0.550653 sec, Total Time: 0.6396 sec
Points: 320,000, Insert Time: 0.8128 sec, Total Search Time: 3.020375 sec, Total Time: 3.8332 sec
Points: 420,000, Insert Time: 0.8010 sec, Total Search Time: 0.348770 sec, Total Time: 1.1498 sec
Points: 430,000, Insert Time: 0.0761 sec, Total Search Time: 0.967903 sec, Total Time: 1.0440 sec
Points: 440,000, Insert Time: 0.0767 sec, Total Search Time: 0.373737 sec, Total Time: 0.4504 sec
Points: 540,000, Insert Time: 0.7973 sec, Total Search Time: 1.148183 sec, Total Time: 1.9455 sec
Points: 1,540,000, Build Time: 9.2189 sec, Total

**------KD Hybrid Log Ratio Run 3-----**

In [27]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.1244 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2352.25it/s]


Total Search Time: 0.014163 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.3948 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 317.96it/s]


Total Search Time: 0.089992 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.556925 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 117234.75it/s]


Insert Time: 0.8562 sec


Querying batch 20: 100%|██████████| 100/100 [00:05<00:00, 19.41it/s]

Total Search Time: 5.866198 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.1244 sec, Total Search Time: 0.014163 sec, Total Time: 0.1386 sec
Points: 110,000, Build Time: 0.3948 sec, Total Search Time: 0.089992 sec, Total Time: 0.4848 sec
Points: 210,000, Insert Time: 0.7734 sec, Total Search Time: 0.148291 sec, Total Time: 0.9217 sec
Points: 220,000, Insert Time: 0.0758 sec, Total Search Time: 0.513656 sec, Total Time: 0.5894 sec
Points: 320,000, Insert Time: 0.8629 sec, Total Search Time: 2.867665 sec, Total Time: 3.7306 sec
Points: 420,000, Insert Time: 0.7802 sec, Total Search Time: 0.349926 sec, Total Time: 1.1301 sec
Points: 430,000, Insert Time: 0.0743 sec, Total Search Time: 0.877420 sec, Total Time: 0.9517 sec
Points: 440,000, Insert Time: 0.0943 sec, Total Search Time: 0.356676 sec, Total Time: 0.4510 sec
Points: 540,000, Insert Time: 0.8024 sec, Total Search Time: 1.081252 sec, Total Time: 1.8837 sec
Points: 1,540,000, Build Time: 9.5790 sec, Total

**--------kd hybrid log ratio analysis-----------**

In [28]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 108.3173',
    'Total Time:: 108.0342',
    'Total Time:: 106.7255'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 107.6923 sec, Std Dev: 0.8492 sec, CV: 0.79%


---------------------------

**----------KD Tree Epsilon Dynamic---------**

In [31]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-----------kd dyn sqrt run 1---------**

In [32]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0260 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2568.39it/s]

Total Search Time: 0.013818 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.9930 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 413.67it/s]


...

Total Search Time: 1.399455 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87463.44it/s]


Insert Time: 1.1470 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 27.15it/s]

Total Search Time: 4.456830 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0260 sec, Total Search Time: 0.013818 sec, Total Time: 0.0398 sec
Points: 110,000, Build Time: 0.9930 sec, Total Search Time: 0.098130 sec, Total Time: 1.0911 sec
Points: 210,000, Insert Time: 0.8888 sec, Total Search Time: 0.160266 sec, Total Time: 1.0490 sec
Points: 220,000, Insert Time: 0.0807 sec, Total Search Time: 0.475293 sec, Total Time: 0.5560 sec
Points: 320,000, Build Time: 2.4251 sec, Total Search Time: 3.247154 sec, Total Time: 5.6723 sec
Points: 420,000, Insert Time: 1.0195 sec, Total Search Time: 0.291843 sec, Total Time: 1.3114 sec
Points: 430,000, Insert Time: 0.1236 sec, Total Search Time: 0.824891 sec, Total Time: 0.9485 sec
Points: 440,000, Insert Time: 0.1284 sec, Total Search Time: 0.322043 sec, Total Time: 0.4504 sec
Points: 540,000, Insert Time: 1.0640 sec, Total Search Time: 0.963310 sec, Total Time: 2.0273 sec
Points: 1,540,000, Build Time: 13.5412 sec, Total

**-----------kd dyn sqrt run 2--------**

In [38]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0283 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2559.22it/s]

Total Search Time: 0.014189 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5223 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 448.09it/s]


...

Total Search Time: 1.483052 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 85480.32it/s]


Insert Time: 1.1736 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 25.37it/s]

Total Search Time: 4.778312 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0283 sec, Total Search Time: 0.014189 sec, Total Time: 0.0425 sec
Points: 110,000, Build Time: 0.5223 sec, Total Search Time: 0.082676 sec, Total Time: 0.6050 sec
Points: 210,000, Insert Time: 0.9716 sec, Total Search Time: 0.172078 sec, Total Time: 1.1436 sec
Points: 220,000, Insert Time: 0.1194 sec, Total Search Time: 0.447394 sec, Total Time: 0.5668 sec
Points: 320,000, Build Time: 3.1368 sec, Total Search Time: 3.540371 sec, Total Time: 6.6772 sec
Points: 420,000, Insert Time: 1.0552 sec, Total Search Time: 0.331695 sec, Total Time: 1.3869 sec
Points: 430,000, Insert Time: 0.1000 sec, Total Search Time: 0.911809 sec, Total Time: 1.0119 sec
Points: 440,000, Insert Time: 0.0973 sec, Total Search Time: 0.309778 sec, Total Time: 0.4071 sec
Points: 540,000, Insert Time: 1.0141 sec, Total Search Time: 1.015362 sec, Total Time: 2.0295 sec
Points: 1,540,000, Build Time: 13.8955 sec, Total

**----------kd dyn sqrt run 3--------**

In [43]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0273 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2448.16it/s]

Total Search Time: 0.014506 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.2892 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 366.64it/s]


...

Total Search Time: 1.554444 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 88230.55it/s]


Insert Time: 1.1365 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 25.49it/s]

Total Search Time: 4.794725 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0273 sec, Total Search Time: 0.014506 sec, Total Time: 0.0419 sec
Points: 110,000, Build Time: 1.2892 sec, Total Search Time: 0.099596 sec, Total Time: 1.3888 sec
Points: 210,000, Insert Time: 0.9188 sec, Total Search Time: 0.189635 sec, Total Time: 1.1085 sec
Points: 220,000, Insert Time: 0.1075 sec, Total Search Time: 0.467091 sec, Total Time: 0.5746 sec
Points: 320,000, Build Time: 2.7231 sec, Total Search Time: 3.523369 sec, Total Time: 6.2464 sec
Points: 420,000, Insert Time: 0.9718 sec, Total Search Time: 0.321006 sec, Total Time: 1.2928 sec
Points: 430,000, Insert Time: 0.0980 sec, Total Search Time: 0.903051 sec, Total Time: 1.0011 sec
Points: 440,000, Insert Time: 0.0971 sec, Total Search Time: 0.341387 sec, Total Time: 0.4384 sec
Points: 540,000, Insert Time: 0.9989 sec, Total Search Time: 1.065194 sec, Total Time: 2.0641 sec
Points: 1,540,000, Build Time: 14.2871 sec, Total

**------------kd dyn sqrt analysis---------**

In [48]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 94.8923',
    'Total Time:: 100.2308',
    'Total Time:: 102.3810'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 99.1680 sec, Std Dev: 3.8558 sec, CV: 3.89%


-----------------------------------------

**-------KD Tree Epsilon Dyn Log Threshold Run 1-----**

In [33]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0241 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2289.90it/s]


Total Search Time: 0.014450 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.9865 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 450.42it/s]


Total Search Time: 0.082343 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.517998 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 78377.08it/s]


Insert Time: 1.2784 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.57it/s]

Total Search Time: 4.563885 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0241 sec, Total Search Time: 0.014450 sec, Total Time: 0.0385 sec
Points: 110,000, Build Time: 0.9865 sec, Total Search Time: 0.082343 sec, Total Time: 1.0688 sec
Points: 210,000, Insert Time: 0.8659 sec, Total Search Time: 0.163923 sec, Total Time: 1.0298 sec
Points: 220,000, Insert Time: 0.0829 sec, Total Search Time: 0.461065 sec, Total Time: 0.5439 sec
Points: 320,000, Insert Time: 0.9407 sec, Total Search Time: 2.532216 sec, Total Time: 3.4729 sec
Points: 420,000, Insert Time: 0.8793 sec, Total Search Time: 0.410848 sec, Total Time: 1.2901 sec
Points: 430,000, Insert Time: 0.1177 sec, Total Search Time: 0.919167 sec, Total Time: 1.0369 sec
Points: 440,000, Insert Time: 0.1493 sec, Total Search Time: 0.438917 sec, Total Time: 0.5882 sec
Points: 540,000, Insert Time: 0.9733 sec, Total Search Time: 1.081895 sec, Total Time: 2.0552 sec
Points: 1,540,000, Build Time: 14.7557 sec, Tota

**-----KD Dyn Run 2--------**

In [39]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0294 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2430.85it/s]


Total Search Time: 0.015644 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.8006 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 433.47it/s]


Total Search Time: 0.082911 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.507608 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 84928.82it/s]


Insert Time: 1.1828 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.52it/s]

Total Search Time: 4.687482 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0294 sec, Total Search Time: 0.015644 sec, Total Time: 0.0450 sec
Points: 110,000, Build Time: 0.8006 sec, Total Search Time: 0.082911 sec, Total Time: 0.8836 sec
Points: 210,000, Insert Time: 0.9458 sec, Total Search Time: 0.179527 sec, Total Time: 1.1253 sec
Points: 220,000, Insert Time: 0.0871 sec, Total Search Time: 0.482445 sec, Total Time: 0.5695 sec
Points: 320,000, Insert Time: 0.9100 sec, Total Search Time: 2.436691 sec, Total Time: 3.3467 sec
Points: 420,000, Insert Time: 0.9522 sec, Total Search Time: 0.366162 sec, Total Time: 1.3184 sec
Points: 430,000, Insert Time: 0.0889 sec, Total Search Time: 0.812542 sec, Total Time: 0.9015 sec
Points: 440,000, Insert Time: 0.0890 sec, Total Search Time: 0.374813 sec, Total Time: 0.4638 sec
Points: 540,000, Insert Time: 0.9785 sec, Total Search Time: 1.007617 sec, Total Time: 1.9861 sec
Points: 1,540,000, Build Time: 15.1514 sec, Tota

**-----KD Dyn Run 3------**

In [44]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0259 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2548.95it/s]


Total Search Time: 0.014362 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5255 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 441.76it/s]


Total Search Time: 0.086075 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.449173 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 92905.64it/s]


Insert Time: 1.0794 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.41it/s]

Total Search Time: 4.624270 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0259 sec, Total Search Time: 0.014362 sec, Total Time: 0.0402 sec
Points: 110,000, Build Time: 0.5255 sec, Total Search Time: 0.086075 sec, Total Time: 0.6116 sec
Points: 210,000, Insert Time: 0.9136 sec, Total Search Time: 0.181802 sec, Total Time: 1.0954 sec
Points: 220,000, Insert Time: 0.0974 sec, Total Search Time: 0.459885 sec, Total Time: 0.5573 sec
Points: 320,000, Insert Time: 0.8833 sec, Total Search Time: 2.320671 sec, Total Time: 3.2039 sec
Points: 420,000, Insert Time: 0.9015 sec, Total Search Time: 0.333153 sec, Total Time: 1.2346 sec
Points: 430,000, Insert Time: 0.0883 sec, Total Search Time: 0.802332 sec, Total Time: 0.8906 sec
Points: 440,000, Insert Time: 0.0866 sec, Total Search Time: 0.369402 sec, Total Time: 0.4560 sec
Points: 540,000, Insert Time: 0.8851 sec, Total Search Time: 0.967541 sec, Total Time: 1.8526 sec
Points: 1,540,000, Build Time: 14.5267 sec, Tota

**-----------kd dyn log analysis-----------**

In [47]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 100.5878',
    'Total Time:: 97.5774',
    'Total Time:: 95.6441'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 97.9364 sec, Std Dev: 2.4913 sec, CV: 2.54%


------------------------------------

**------KD Tree Dynamic Log Ratio Run 1--------**

In [34]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0250 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2381.84it/s]


Total Search Time: 0.014841 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5743 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 448.61it/s]


Total Search Time: 0.082584 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.630283 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87484.71it/s]


Insert Time: 1.1463 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 25.26it/s]

Total Search Time: 4.875948 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0250 sec, Total Search Time: 0.014841 sec, Total Time: 0.0399 sec
Points: 110,000, Build Time: 0.5743 sec, Total Search Time: 0.082584 sec, Total Time: 0.6569 sec
Points: 210,000, Insert Time: 0.9524 sec, Total Search Time: 0.180992 sec, Total Time: 1.1334 sec
Points: 220,000, Insert Time: 0.1027 sec, Total Search Time: 0.499095 sec, Total Time: 0.6018 sec
Points: 320,000, Insert Time: 0.9794 sec, Total Search Time: 2.467670 sec, Total Time: 3.4471 sec
Points: 420,000, Insert Time: 0.9051 sec, Total Search Time: 0.367811 sec, Total Time: 1.2729 sec
Points: 430,000, Insert Time: 0.0831 sec, Total Search Time: 0.764349 sec, Total Time: 0.8474 sec
Points: 440,000, Insert Time: 0.0873 sec, Total Search Time: 0.355086 sec, Total Time: 0.4424 sec
Points: 540,000, Insert Time: 0.8955 sec, Total Search Time: 0.999298 sec, Total Time: 1.8948 sec
Points: 1,540,000, Build Time: 15.3489 sec, Tota

**--------KD Dyn Log Ratio Run 2------**

In [40]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0282 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2344.82it/s]


Total Search Time: 0.014208 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.2340 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 419.33it/s]


Total Search Time: 0.100384 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.684326 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 84318.84it/s]


Insert Time: 1.1907 sec


Querying batch 20: 100%|██████████| 100/100 [00:04<00:00, 24.26it/s]

Total Search Time: 5.163028 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0282 sec, Total Search Time: 0.014208 sec, Total Time: 0.0424 sec
Points: 110,000, Build Time: 1.2340 sec, Total Search Time: 0.100384 sec, Total Time: 1.3344 sec
Points: 210,000, Insert Time: 0.8549 sec, Total Search Time: 0.193261 sec, Total Time: 1.0482 sec
Points: 220,000, Insert Time: 0.1124 sec, Total Search Time: 0.512026 sec, Total Time: 0.6244 sec
Points: 320,000, Insert Time: 0.8720 sec, Total Search Time: 2.521943 sec, Total Time: 3.3939 sec
Points: 420,000, Insert Time: 0.9133 sec, Total Search Time: 0.381608 sec, Total Time: 1.2949 sec
Points: 430,000, Insert Time: 0.1170 sec, Total Search Time: 0.882617 sec, Total Time: 0.9996 sec
Points: 440,000, Insert Time: 0.0923 sec, Total Search Time: 0.391777 sec, Total Time: 0.4841 sec
Points: 540,000, Insert Time: 0.9152 sec, Total Search Time: 1.016518 sec, Total Time: 1.9317 sec
Points: 1,540,000, Build Time: 15.2980 sec, Tota

**-------KD Dynamic Log Ratio Run 3----**

In [45]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0319 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2384.55it/s]


Total Search Time: 0.015040 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5089 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 445.03it/s]


Total Search Time: 0.081846 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.507914 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 83923.53it/s]


Insert Time: 1.1967 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 25.12it/s]

Total Search Time: 4.888087 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0319 sec, Total Search Time: 0.015040 sec, Total Time: 0.0470 sec
Points: 110,000, Build Time: 0.5089 sec, Total Search Time: 0.081846 sec, Total Time: 0.5908 sec
Points: 210,000, Insert Time: 0.9627 sec, Total Search Time: 0.177755 sec, Total Time: 1.1404 sec
Points: 220,000, Insert Time: 0.0969 sec, Total Search Time: 0.453580 sec, Total Time: 0.5505 sec
Points: 320,000, Insert Time: 0.9701 sec, Total Search Time: 2.289535 sec, Total Time: 3.2597 sec
Points: 420,000, Insert Time: 0.9765 sec, Total Search Time: 0.361088 sec, Total Time: 1.3376 sec
Points: 430,000, Insert Time: 0.0876 sec, Total Search Time: 0.786478 sec, Total Time: 0.8741 sec
Points: 440,000, Insert Time: 0.1422 sec, Total Search Time: 0.357797 sec, Total Time: 0.5000 sec
Points: 540,000, Insert Time: 0.9045 sec, Total Search Time: 0.941464 sec, Total Time: 1.8460 sec
Points: 1,540,000, Build Time: 15.0701 sec, Tota

**----------kd dyn log ratio analysis-----------------**

In [46]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 99.2301',
    'Total Time:: 100.8397',
    'Total Time:: 96.4933'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 98.8544 sec, Std Dev: 2.1974 sec, CV: 2.22%


---------------------------

**----------VP Tree Epsilon Opt Parameters------**

In [49]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**---------vp hybrid sqrt run 1---------**

In [50]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0161 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3350.08it/s]

Total Search Time: 0.010230 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.2912 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1260.88it/s]


...

Total Search Time: 0.378387 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57528.64it/s]


Insert Time: 1.7407 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 127.27it/s]

Total Search Time: 1.069399 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0161 sec, Total Search Time: 0.010230 sec, Total Time: 0.0263 sec
Points: 110,000, Build Time: 0.2912 sec, Total Search Time: 0.023301 sec, Total Time: 0.3145 sec
Points: 210,000, Insert Time: 1.3634 sec, Total Search Time: 0.069422 sec, Total Time: 1.4328 sec
Points: 220,000, Insert Time: 0.1403 sec, Total Search Time: 0.205876 sec, Total Time: 0.3462 sec
Points: 320,000, Build Time: 1.0960 sec, Total Search Time: 0.369272 sec, Total Time: 1.4652 sec
Points: 420,000, Insert Time: 1.4491 sec, Total Search Time: 0.084904 sec, Total Time: 1.5340 sec
Points: 430,000, Insert Time: 0.1515 sec, Total Search Time: 0.222514 sec, Total Time: 0.3740 sec
Points: 440,000, Insert Time: 0.1510 sec, Total Search Time: 0.093087 sec, Total Time: 0.2441 sec
Points: 540,000, Insert Time: 1.4776 sec, Total Search Time: 0.398826 sec, Total Time: 1.8764 sec
Points: 1,540,000, Build Time: 6.5274 sec, Total 

**--------vp hybrid sqrt run 2--------**

In [56]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.1464 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3368.78it/s]

Total Search Time: 0.010105 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.3201 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1566.79it/s]


...

Total Search Time: 0.397713 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59420.46it/s]


Insert Time: 1.6853 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 143.28it/s]

Total Search Time: 1.008646 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.1464 sec, Total Search Time: 0.010105 sec, Total Time: 0.1565 sec
Points: 110,000, Build Time: 0.3201 sec, Total Search Time: 0.022512 sec, Total Time: 0.3426 sec
Points: 210,000, Insert Time: 1.3773 sec, Total Search Time: 0.082883 sec, Total Time: 1.4602 sec
Points: 220,000, Insert Time: 0.1381 sec, Total Search Time: 0.223339 sec, Total Time: 0.3614 sec
Points: 320,000, Build Time: 1.4210 sec, Total Search Time: 0.236587 sec, Total Time: 1.6576 sec
Points: 420,000, Insert Time: 1.6201 sec, Total Search Time: 0.086965 sec, Total Time: 1.7071 sec
Points: 430,000, Insert Time: 0.1644 sec, Total Search Time: 0.188112 sec, Total Time: 0.3525 sec
Points: 440,000, Insert Time: 0.1726 sec, Total Search Time: 0.098881 sec, Total Time: 0.2715 sec
Points: 540,000, Insert Time: 1.6324 sec, Total Search Time: 0.394243 sec, Total Time: 2.0266 sec
Points: 1,540,000, Build Time: 7.4504 sec, Total 

**----------vp hybrid sqrt run 3----------**

In [61]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0227 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3103.10it/s]

Total Search Time: 0.011201 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4389 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1281.11it/s]


...

Total Search Time: 0.370716 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57686.03it/s]


Insert Time: 1.7352 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 126.30it/s]

Total Search Time: 1.069957 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0227 sec, Total Search Time: 0.011201 sec, Total Time: 0.0339 sec
Points: 110,000, Build Time: 0.4389 sec, Total Search Time: 0.022462 sec, Total Time: 0.4614 sec
Points: 210,000, Insert Time: 1.3567 sec, Total Search Time: 0.078521 sec, Total Time: 1.4352 sec
Points: 220,000, Insert Time: 0.1445 sec, Total Search Time: 0.211165 sec, Total Time: 0.3557 sec
Points: 320,000, Build Time: 1.2737 sec, Total Search Time: 0.377676 sec, Total Time: 1.6514 sec
Points: 420,000, Insert Time: 1.4349 sec, Total Search Time: 0.080195 sec, Total Time: 1.5151 sec
Points: 430,000, Insert Time: 0.1459 sec, Total Search Time: 0.216574 sec, Total Time: 0.3624 sec
Points: 440,000, Insert Time: 0.1508 sec, Total Search Time: 0.094298 sec, Total Time: 0.2451 sec
Points: 540,000, Insert Time: 1.4669 sec, Total Search Time: 0.403371 sec, Total Time: 1.8703 sec
Points: 1,540,000, Build Time: 6.7368 sec, Total 

**---------vp hybrid sqrt analysis---------**

In [66]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 32.7306',
    'Total Time:: 32.7856',
    'Total Time:: 33.1111'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 32.8758 sec, Std Dev: 0.2057 sec, CV: 0.63%


------------------------------------

**-----VP hybrid Log Threshold Run 1------**

In [51]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0184 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3198.10it/s]


Total Search Time: 0.010298 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.6297 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1155.47it/s]


Total Search Time: 0.024718 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.371290 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57506.56it/s]


Insert Time: 1.7411 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 129.36it/s]

Total Search Time: 1.060984 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0184 sec, Total Search Time: 0.010298 sec, Total Time: 0.0287 sec
Points: 110,000, Build Time: 0.6297 sec, Total Search Time: 0.024718 sec, Total Time: 0.6544 sec
Points: 210,000, Insert Time: 1.3543 sec, Total Search Time: 0.081535 sec, Total Time: 1.4358 sec
Points: 220,000, Insert Time: 0.1430 sec, Total Search Time: 0.218180 sec, Total Time: 0.3612 sec
Points: 320,000, Insert Time: 1.3726 sec, Total Search Time: 1.594708 sec, Total Time: 2.9673 sec
Points: 420,000, Insert Time: 1.3992 sec, Total Search Time: 0.246360 sec, Total Time: 1.6455 sec
Points: 430,000, Insert Time: 0.1474 sec, Total Search Time: 0.567011 sec, Total Time: 0.7144 sec
Points: 440,000, Insert Time: 0.1388 sec, Total Search Time: 0.237136 sec, Total Time: 0.3760 sec
Points: 540,000, Insert Time: 1.3782 sec, Total Search Time: 0.783751 sec, Total Time: 2.1619 sec
Points: 1,540,000, Build Time: 6.5586 sec, Total

**-------VP Hybrid Run 2-----**

In [57]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0199 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3291.97it/s]


Total Search Time: 0.009988 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4641 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1509.67it/s]


Total Search Time: 0.024448 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.406221 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59391.42it/s]


Insert Time: 1.6858 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 142.65it/s]

Total Search Time: 1.014167 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0199 sec, Total Search Time: 0.009988 sec, Total Time: 0.0299 sec
Points: 110,000, Build Time: 0.4641 sec, Total Search Time: 0.024448 sec, Total Time: 0.4885 sec
Points: 210,000, Insert Time: 1.3683 sec, Total Search Time: 0.080938 sec, Total Time: 1.4492 sec
Points: 220,000, Insert Time: 0.1416 sec, Total Search Time: 0.217981 sec, Total Time: 0.3596 sec
Points: 320,000, Insert Time: 1.3746 sec, Total Search Time: 1.410952 sec, Total Time: 2.7856 sec
Points: 420,000, Insert Time: 1.3752 sec, Total Search Time: 0.253204 sec, Total Time: 1.6284 sec
Points: 430,000, Insert Time: 0.1385 sec, Total Search Time: 0.547516 sec, Total Time: 0.6860 sec
Points: 440,000, Insert Time: 0.1405 sec, Total Search Time: 0.258324 sec, Total Time: 0.3989 sec
Points: 540,000, Insert Time: 1.3679 sec, Total Search Time: 0.739055 sec, Total Time: 2.1070 sec
Points: 1,540,000, Build Time: 7.4428 sec, Total

**-----VP Hybrid Run 3--------**

In [62]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0136 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3399.09it/s]


Total Search Time: 0.009469 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4294 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1267.64it/s]


Total Search Time: 0.022167 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.375128 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57625.12it/s]


Insert Time: 1.7385 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 128.76it/s]

Total Search Time: 1.053690 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0136 sec, Total Search Time: 0.009469 sec, Total Time: 0.0231 sec
Points: 110,000, Build Time: 0.4294 sec, Total Search Time: 0.022167 sec, Total Time: 0.4515 sec
Points: 210,000, Insert Time: 1.3698 sec, Total Search Time: 0.072815 sec, Total Time: 1.4426 sec
Points: 220,000, Insert Time: 0.1426 sec, Total Search Time: 0.210163 sec, Total Time: 0.3528 sec
Points: 320,000, Insert Time: 1.3778 sec, Total Search Time: 1.619560 sec, Total Time: 2.9974 sec
Points: 420,000, Insert Time: 1.3845 sec, Total Search Time: 0.237622 sec, Total Time: 1.6221 sec
Points: 430,000, Insert Time: 0.1411 sec, Total Search Time: 0.565763 sec, Total Time: 0.7068 sec
Points: 440,000, Insert Time: 0.1452 sec, Total Search Time: 0.246539 sec, Total Time: 0.3918 sec
Points: 540,000, Insert Time: 1.3809 sec, Total Search Time: 0.806739 sec, Total Time: 2.1876 sec
Points: 1,540,000, Build Time: 6.6332 sec, Total

**---------vp hybrid log analysis------------**

In [65]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 35.3430',
    'Total Time:: 34.4916',
    'Total Time:: 35.1720'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 35.0022 sec, Std Dev: 0.4504 sec, CV: 1.29%


--------------------------------

**-----VP Tree Epsilon Log Ratio Run 1-------**

In [52]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0176 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3227.50it/s]


Total Search Time: 0.010514 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4109 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1299.35it/s]


Total Search Time: 0.023568 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.382225 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57478.68it/s]


Insert Time: 1.7427 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 131.57it/s]

Total Search Time: 1.046250 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0176 sec, Total Search Time: 0.010514 sec, Total Time: 0.0282 sec
Points: 110,000, Build Time: 0.4109 sec, Total Search Time: 0.023568 sec, Total Time: 0.4345 sec
Points: 210,000, Insert Time: 1.3510 sec, Total Search Time: 0.078505 sec, Total Time: 1.4295 sec
Points: 220,000, Insert Time: 0.1410 sec, Total Search Time: 0.208659 sec, Total Time: 0.3496 sec
Points: 320,000, Insert Time: 1.3580 sec, Total Search Time: 1.554466 sec, Total Time: 2.9125 sec
Points: 420,000, Insert Time: 1.3869 sec, Total Search Time: 0.242981 sec, Total Time: 1.6299 sec
Points: 430,000, Insert Time: 0.1419 sec, Total Search Time: 0.558824 sec, Total Time: 0.7007 sec
Points: 440,000, Insert Time: 0.1379 sec, Total Search Time: 0.237832 sec, Total Time: 0.3757 sec
Points: 540,000, Insert Time: 1.3563 sec, Total Search Time: 0.775073 sec, Total Time: 2.1314 sec
Points: 1,540,000, Build Time: 6.4679 sec, Total

**------VP Hybrid Log Ratio Run 2---------**

In [58]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0121 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3239.72it/s]


Total Search Time: 0.009997 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4618 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1501.80it/s]


Total Search Time: 0.024204 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.394725 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59380.40it/s]


Insert Time: 1.6864 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 143.42it/s]

Total Search Time: 1.008357 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0121 sec, Total Search Time: 0.009997 sec, Total Time: 0.0220 sec
Points: 110,000, Build Time: 0.4618 sec, Total Search Time: 0.024204 sec, Total Time: 0.4860 sec
Points: 210,000, Insert Time: 1.3618 sec, Total Search Time: 0.090063 sec, Total Time: 1.4519 sec
Points: 220,000, Insert Time: 0.1385 sec, Total Search Time: 0.215108 sec, Total Time: 0.3536 sec
Points: 320,000, Insert Time: 1.3826 sec, Total Search Time: 1.448452 sec, Total Time: 2.8311 sec
Points: 420,000, Insert Time: 1.3642 sec, Total Search Time: 0.250497 sec, Total Time: 1.6147 sec
Points: 430,000, Insert Time: 0.1405 sec, Total Search Time: 0.546437 sec, Total Time: 0.6869 sec
Points: 440,000, Insert Time: 0.1398 sec, Total Search Time: 0.263850 sec, Total Time: 0.4036 sec
Points: 540,000, Insert Time: 1.3503 sec, Total Search Time: 0.728887 sec, Total Time: 2.0792 sec
Points: 1,540,000, Build Time: 7.3440 sec, Total

**------VP Hybrid Log Ratio Run 3----------**

In [63]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0199 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3271.05it/s]


Total Search Time: 0.010227 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4343 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 1256.51it/s]


Total Search Time: 0.022724 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.384586 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57777.10it/s]


Insert Time: 1.7331 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 130.64it/s]

Total Search Time: 1.030008 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0199 sec, Total Search Time: 0.010227 sec, Total Time: 0.0302 sec
Points: 110,000, Build Time: 0.4343 sec, Total Search Time: 0.022724 sec, Total Time: 0.4571 sec
Points: 210,000, Insert Time: 1.3638 sec, Total Search Time: 0.083062 sec, Total Time: 1.4469 sec
Points: 220,000, Insert Time: 0.1400 sec, Total Search Time: 0.216042 sec, Total Time: 0.3561 sec
Points: 320,000, Insert Time: 1.3673 sec, Total Search Time: 1.578070 sec, Total Time: 2.9453 sec
Points: 420,000, Insert Time: 1.3670 sec, Total Search Time: 0.238861 sec, Total Time: 1.6059 sec
Points: 430,000, Insert Time: 0.1401 sec, Total Search Time: 0.579024 sec, Total Time: 0.7191 sec
Points: 440,000, Insert Time: 0.1416 sec, Total Search Time: 0.243644 sec, Total Time: 0.3853 sec
Points: 540,000, Insert Time: 1.3806 sec, Total Search Time: 0.775373 sec, Total Time: 2.1560 sec
Points: 1,540,000, Build Time: 6.6261 sec, Total

**---------vp hybrid log ratio analysis--------**

In [64]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 34.8203',
    'Total Time:: 34.2491',
    'Total Time:: 35.0064'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 34.6919 sec, Std Dev: 0.3946 sec, CV: 1.14%


--------------------------

**-------VP Tree Epsilon Dynamic-------**

In [67]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

**-------vp dyn sqrt run 1-----------**

In [68]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0192 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2943.89it/s]

Total Search Time: 0.011772 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.5183 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5239.28it/s]


...

Total Search Time: 0.459970 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 82462.97it/s]


Insert Time: 1.2151 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 100.65it/s]

Total Search Time: 1.349541 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0192 sec, Total Search Time: 0.011772 sec, Total Time: 0.0309 sec
Points: 110,000, Build Time: 0.5183 sec, Total Search Time: 0.006998 sec, Total Time: 0.5253 sec
Points: 210,000, Insert Time: 1.2368 sec, Total Search Time: 0.076580 sec, Total Time: 1.3134 sec
Points: 220,000, Insert Time: 0.1226 sec, Total Search Time: 0.239660 sec, Total Time: 0.3623 sec
Points: 320,000, Build Time: 1.6059 sec, Total Search Time: 0.024719 sec, Total Time: 1.6307 sec
Points: 420,000, Insert Time: 1.2870 sec, Total Search Time: 0.087101 sec, Total Time: 1.3741 sec
Points: 430,000, Insert Time: 0.1340 sec, Total Search Time: 0.215052 sec, Total Time: 0.3491 sec
Points: 440,000, Insert Time: 0.1262 sec, Total Search Time: 0.107311 sec, Total Time: 0.2335 sec
Points: 540,000, Insert Time: 1.3134 sec, Total Search Time: 0.447726 sec, Total Time: 1.7611 sec
Points: 1,540,000, Build Time: 5.5251 sec, Total 

**---------vp dyn sqrt run 2---------**

In [74]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0262 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3052.07it/s]

Total Search Time: 0.010639 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.5787 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5514.11it/s]


...

Total Search Time: 0.458312 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69861.53it/s]


Insert Time: 1.4340 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 136.12it/s]

Total Search Time: 1.126461 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0262 sec, Total Search Time: 0.010639 sec, Total Time: 0.0368 sec
Points: 110,000, Build Time: 0.5787 sec, Total Search Time: 0.008153 sec, Total Time: 0.5869 sec
Points: 210,000, Insert Time: 1.2908 sec, Total Search Time: 0.083268 sec, Total Time: 1.3740 sec
Points: 220,000, Insert Time: 0.1321 sec, Total Search Time: 0.209143 sec, Total Time: 0.3413 sec
Points: 320,000, Build Time: 1.0473 sec, Total Search Time: 0.096813 sec, Total Time: 1.1441 sec
Points: 420,000, Insert Time: 1.3148 sec, Total Search Time: 0.080684 sec, Total Time: 1.3954 sec
Points: 430,000, Insert Time: 0.1480 sec, Total Search Time: 0.201024 sec, Total Time: 0.3490 sec
Points: 440,000, Insert Time: 0.1435 sec, Total Search Time: 0.111632 sec, Total Time: 0.2551 sec
Points: 540,000, Insert Time: 1.3101 sec, Total Search Time: 0.386599 sec, Total Time: 1.6967 sec
Points: 1,540,000, Build Time: 6.0976 sec, Total 

**---------vp dyn sqrt run 3--------**

In [79]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0227 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3255.56it/s]

Total Search Time: 0.009785 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4340 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 4211.78it/s]


...

Total Search Time: 0.458738 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 77500.37it/s]


Insert Time: 1.2932 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 104.30it/s]

Total Search Time: 1.315196 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0227 sec, Total Search Time: 0.009785 sec, Total Time: 0.0325 sec
Points: 110,000, Build Time: 0.4340 sec, Total Search Time: 0.010011 sec, Total Time: 0.4440 sec
Points: 210,000, Insert Time: 1.2259 sec, Total Search Time: 0.095313 sec, Total Time: 1.3213 sec
Points: 220,000, Insert Time: 0.1255 sec, Total Search Time: 0.232457 sec, Total Time: 0.3580 sec
Points: 320,000, Build Time: 1.0296 sec, Total Search Time: 0.027435 sec, Total Time: 1.0570 sec
Points: 420,000, Insert Time: 1.3142 sec, Total Search Time: 0.083649 sec, Total Time: 1.3979 sec
Points: 430,000, Insert Time: 0.1352 sec, Total Search Time: 0.217564 sec, Total Time: 0.3527 sec
Points: 440,000, Insert Time: 0.1260 sec, Total Search Time: 0.093065 sec, Total Time: 0.2191 sec
Points: 540,000, Insert Time: 1.2867 sec, Total Search Time: 0.441097 sec, Total Time: 1.7278 sec
Points: 1,540,000, Build Time: 5.5754 sec, Total 

**--------vp dyn sqrt analysis---------**

In [84]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 29.4651',
    'Total Time:: 29.2336',
    'Total Time:: 28.8478'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 29.1822 sec, Std Dev: 0.3118 sec, CV: 1.07%


------------------------------

**-----VP Epsilon Dynamic Log Threshold Run 1------**

In [69]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0264 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3232.85it/s]


Total Search Time: 0.010007 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4063 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5055.81it/s]


Total Search Time: 0.007793 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.445951 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 81842.84it/s]


Insert Time: 1.2236 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 102.51it/s]

Total Search Time: 1.326956 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0264 sec, Total Search Time: 0.010007 sec, Total Time: 0.0364 sec
Points: 110,000, Build Time: 0.4063 sec, Total Search Time: 0.007793 sec, Total Time: 0.4141 sec
Points: 210,000, Insert Time: 1.2180 sec, Total Search Time: 0.084897 sec, Total Time: 1.3029 sec
Points: 220,000, Insert Time: 0.1279 sec, Total Search Time: 0.233698 sec, Total Time: 0.3616 sec
Points: 320,000, Insert Time: 1.2714 sec, Total Search Time: 1.794687 sec, Total Time: 3.0661 sec
Points: 420,000, Insert Time: 1.2493 sec, Total Search Time: 0.273313 sec, Total Time: 1.5226 sec
Points: 430,000, Insert Time: 0.1223 sec, Total Search Time: 0.671000 sec, Total Time: 0.7933 sec
Points: 440,000, Insert Time: 0.1267 sec, Total Search Time: 0.297082 sec, Total Time: 0.4238 sec
Points: 540,000, Insert Time: 1.3116 sec, Total Search Time: 0.878348 sec, Total Time: 2.1899 sec
Points: 1,540,000, Build Time: 5.6052 sec, Total

**--------VP Dynamic Run 2--------**

In [75]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0243 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2647.50it/s]


Total Search Time: 0.012369 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.6898 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5193.54it/s]


Total Search Time: 0.008404 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.455231 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69810.38it/s]


Insert Time: 1.4351 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 142.30it/s]

Total Search Time: 1.076666 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0243 sec, Total Search Time: 0.012369 sec, Total Time: 0.0367 sec
Points: 110,000, Build Time: 0.6898 sec, Total Search Time: 0.008404 sec, Total Time: 0.6982 sec
Points: 210,000, Insert Time: 1.3339 sec, Total Search Time: 0.091852 sec, Total Time: 1.4258 sec
Points: 220,000, Insert Time: 0.1307 sec, Total Search Time: 0.233842 sec, Total Time: 0.3645 sec
Points: 320,000, Insert Time: 1.2977 sec, Total Search Time: 1.531210 sec, Total Time: 2.8289 sec
Points: 420,000, Insert Time: 1.2670 sec, Total Search Time: 0.253995 sec, Total Time: 1.5210 sec
Points: 430,000, Insert Time: 0.1316 sec, Total Search Time: 0.616973 sec, Total Time: 0.7486 sec
Points: 440,000, Insert Time: 0.1282 sec, Total Search Time: 0.283943 sec, Total Time: 0.4121 sec
Points: 540,000, Insert Time: 1.3061 sec, Total Search Time: 0.814314 sec, Total Time: 2.1205 sec
Points: 1,540,000, Build Time: 6.2355 sec, Total

**------VP Dynamic Run 3----------**

In [80]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0263 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3124.48it/s]


Total Search Time: 0.010686 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4069 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5495.68it/s]


Total Search Time: 0.007444 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.470323 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 81476.34it/s]


Insert Time: 1.2311 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 97.87it/s]

Total Search Time: 1.399732 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0263 sec, Total Search Time: 0.010686 sec, Total Time: 0.0370 sec
Points: 110,000, Build Time: 0.4069 sec, Total Search Time: 0.007444 sec, Total Time: 0.4143 sec
Points: 210,000, Insert Time: 1.2229 sec, Total Search Time: 0.096517 sec, Total Time: 1.3194 sec
Points: 220,000, Insert Time: 0.1252 sec, Total Search Time: 0.229300 sec, Total Time: 0.3545 sec
Points: 320,000, Insert Time: 1.2754 sec, Total Search Time: 1.794506 sec, Total Time: 3.0699 sec
Points: 420,000, Insert Time: 1.1927 sec, Total Search Time: 0.278668 sec, Total Time: 1.4714 sec
Points: 430,000, Insert Time: 0.1257 sec, Total Search Time: 0.663966 sec, Total Time: 0.7896 sec
Points: 440,000, Insert Time: 0.1294 sec, Total Search Time: 0.301110 sec, Total Time: 0.4305 sec
Points: 540,000, Insert Time: 1.3185 sec, Total Search Time: 0.910660 sec, Total Time: 2.2292 sec
Points: 1,540,000, Build Time: 5.6074 sec, Total

**---------vp dyn log analysis----------**

In [83]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 32.0172',
    'Total Time:: 32.1619',
    'Total Time:: 32.2749'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 32.1513 sec, Std Dev: 0.1292 sec, CV: 0.40%


------------------------

**-----VP Epsilon Dynamic Log Ratio Run 1-----**

In [70]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.1521 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3044.65it/s]


Total Search Time: 0.011187 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4111 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5725.23it/s]


Total Search Time: 0.006852 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.469843 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 79412.29it/s]


Insert Time: 1.2616 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 99.60it/s]

Total Search Time: 1.358037 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.1521 sec, Total Search Time: 0.011187 sec, Total Time: 0.1633 sec
Points: 110,000, Build Time: 0.4111 sec, Total Search Time: 0.006852 sec, Total Time: 0.4179 sec
Points: 210,000, Insert Time: 1.2221 sec, Total Search Time: 0.091021 sec, Total Time: 1.3131 sec
Points: 220,000, Insert Time: 0.1224 sec, Total Search Time: 0.226842 sec, Total Time: 0.3493 sec
Points: 320,000, Insert Time: 1.2773 sec, Total Search Time: 1.779527 sec, Total Time: 3.0569 sec
Points: 420,000, Insert Time: 1.2137 sec, Total Search Time: 0.269629 sec, Total Time: 1.4833 sec
Points: 430,000, Insert Time: 0.1272 sec, Total Search Time: 0.677359 sec, Total Time: 0.8045 sec
Points: 440,000, Insert Time: 0.1249 sec, Total Search Time: 0.295243 sec, Total Time: 0.4202 sec
Points: 540,000, Insert Time: 1.3025 sec, Total Search Time: 0.902527 sec, Total Time: 2.2050 sec
Points: 1,540,000, Build Time: 5.4305 sec, Total

**--------VP Dyn Log Ratio Run 2-------**

In [76]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.1788 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3130.66it/s]

Total Search Time: 0.011203 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.2839 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5091.41it/s]


...

Total Search Time: 0.450560 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70224.25it/s]


Insert Time: 1.4258 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 137.16it/s]

Total Search Time: 1.114770 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.1788 sec, Total Search Time: 0.011203 sec, Total Time: 0.1900 sec
Points: 110,000, Build Time: 0.2839 sec, Total Search Time: 0.008637 sec, Total Time: 0.2925 sec
Points: 210,000, Insert Time: 1.2875 sec, Total Search Time: 0.095889 sec, Total Time: 1.3834 sec
Points: 220,000, Insert Time: 0.1280 sec, Total Search Time: 0.226302 sec, Total Time: 0.3543 sec
Points: 320,000, Insert Time: 1.3411 sec, Total Search Time: 1.552476 sec, Total Time: 2.8935 sec
Points: 420,000, Insert Time: 1.2732 sec, Total Search Time: 0.267771 sec, Total Time: 1.5410 sec
Points: 430,000, Insert Time: 0.1285 sec, Total Search Time: 0.614031 sec, Total Time: 0.7426 sec
Points: 440,000, Insert Time: 0.1274 sec, Total Search Time: 0.286787 sec, Total Time: 0.4142 sec
Points: 540,000, Insert Time: 1.3012 sec, Total Search Time: 0.824310 sec, Total Time: 2.1255 sec
Points: 1,540,000, Build Time: 6.3294 sec, Total

**-------VP Dyn Log Ratio Run 3--------**

In [81]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0266 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3032.43it/s]


Total Search Time: 0.010777 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.4199 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 5052.16it/s]


Total Search Time: 0.007771 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.463203 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 82058.96it/s]


Insert Time: 1.2214 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 103.32it/s]

Total Search Time: 1.329594 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0266 sec, Total Search Time: 0.010777 sec, Total Time: 0.0374 sec
Points: 110,000, Build Time: 0.4199 sec, Total Search Time: 0.007771 sec, Total Time: 0.4277 sec
Points: 210,000, Insert Time: 1.2181 sec, Total Search Time: 0.083281 sec, Total Time: 1.3014 sec
Points: 220,000, Insert Time: 0.1223 sec, Total Search Time: 0.236744 sec, Total Time: 0.3590 sec
Points: 320,000, Insert Time: 1.2669 sec, Total Search Time: 1.780976 sec, Total Time: 3.0479 sec
Points: 420,000, Insert Time: 1.2133 sec, Total Search Time: 0.268069 sec, Total Time: 1.4814 sec
Points: 430,000, Insert Time: 0.1243 sec, Total Search Time: 0.666115 sec, Total Time: 0.7904 sec
Points: 440,000, Insert Time: 0.1241 sec, Total Search Time: 0.310497 sec, Total Time: 0.4346 sec
Points: 540,000, Insert Time: 1.2911 sec, Total Search Time: 0.911235 sec, Total Time: 2.2023 sec
Points: 1,540,000, Build Time: 5.5547 sec, Total

**--------vp dyn log ratio analysis---------**

In [82]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 31.8224',
    'Total Time:: 31.9644',
    'Total Time:: 31.8786'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 31.8885 sec, Std Dev: 0.0715 sec, CV: 0.22%
